In [ ]:
!pip install finlab_crypto > log.txt
!pip install talib-binary > log.txt

In [ ]:
import finlab_crypto
finlab_crypto.setup()
ohlcv = finlab_crypto.crawler.get_all_binance('BTCUSDT', '1h')

# MACD Strategy

## View Talib function

In [ ]:
import talib

# show how to use talib.MACD
help(talib.MACD)

## Apply Talib function

[isinstance api reference](https://www.w3schools.com/python/ref_func_isinstance.asp):check type

In [ ]:
# only use masdhist ,choose [2]
macdhist = talib.MACD(ohlcv.close, fastperiod=12, slowperiod=26, signalperiod=9)[2]
macdhist.loc['2020-10'].plot()
ohlcv.close.loc['2020-10'].plot(secondary_y=True)

In [ ]:
# see all talib technical func
#talib.get_functions()

In [ ]:
import numbers
from finlab_crypto import Strategy

# generate crossover signal
def crossover(s1, s2):
  # when s1=time_series and s2=constant(like 0)
  if isinstance(s2, numbers.Number):
    return (s1 > s2) & (s1.shift() < s2)
  # when s1 and s2 are time_series
  return (s1 > s2) & (s1.shift() < s2.shift())

# generate crossunder signal
def crossunder(s1, s2):
  # when s1=time_series and s2=constant(like 0)
  if isinstance(s2, numbers.Number):
    return (s1 < s2) & (s1.shift() > s2)
  # when s1 and s2 are time_series
  return (s1 < s2) & (s1.shift() > s2.shift())

# macd strategy
@Strategy(fastperiod=12, slowperiod=26, signalperiod=9)
def macd_strategy(ohlcv):
    macdhist = talib.MACD(
        ohlcv.close,
        fastperiod=macd_strategy.fastperiod,
        slowperiod=macd_strategy.slowperiod,
        signalperiod=macd_strategy.signalperiod)[2]

    entries = crossover(macdhist, 0)
    exits = crossunder(macdhist, 0)
    figures = {'figures':{'macdhist':macdhist}}
    return entries, exits, figures

# Diff:variables={} ,not None in vedio
macd_strategy.backtest(ohlcv, variables={},freq='1h', plot=True)

# Talib Wrapper

[lambda api reference](https://www.w3schools.com/python/python_lambda.asp)

### declare parameters:

lambda ohlcv, macd

### output

crossover(macd.macdhist, 0)




In [ ]:
help(talib.RSI)
# TalibStrategy??

In [ ]:
from finlab_crypto.talib_strategy import TalibStrategy

# def entry_func(ohlcv,macd):
#   return crossover(macd.macdhist,0)
# def exit_func(ohlcv,macd):
#   return crossunder(macd.macdhist,0)


macd_strategy = TalibStrategy(
    'MACD',
    # ohlcv is required, we need it to generate macd
    # entries = ret.entries(ohlcv, o)
    # exits = ret.exits(ohlcv, o)
    lambda ohlcv, macd: crossover(macd.macdhist, 0),
    lambda ohlcv, macd: crossunder(macd.macdhist, 0),
)

#rsi_strategy = TalibStrategy(
#    'RSI'
#    lambda ohlcv, rsi: rsi > 50,
#    lambda ohlcv, rsi: rsi < 50,
#    )

# Diff:variables={} ,not None in vedio
macd_strategy.backtest(ohlcv, variables={}, freq='1h', plot=True)

# Talib Strategy Wrapper Exercise

In [ ]:
help(talib.BBANDS)

In [ ]:
# Build a bband breaout strategy
bbands_strategy = TalibStrategy(
    'BBANDS',
    lambda ohlcv, bb: crossover(ohlcv.close, bb.upperband),
    lambda ohlcv, bb: crossunder(ohlcv.close, bb.lowerband),
    )

bbands_strategy.backtest(ohlcv, variables={'timeperiod': 50}, freq='1h', plot=True)

# Talib Filter Wrapper

In [ ]:
help(talib.LINEARREG_ANGLE)

talib.LINEARREG_ANGLE(ohlcv.close, timeperiod=14).plot()

In [ ]:
from finlab_crypto.talib_filter import TalibFilter

line_angle_filter = TalibFilter(
    'LINEARREG_ANGLE',
    lambda ohlcv, angle: angle > 0
)

In [ ]:
f14 = line_angle_filter.create({'timeperiod': 14})

# Combine talib strategy and filter

In [ ]:
# macd_strategy
# line_angle_filter
from finlab_crypto.talib_strategy import TalibStrategy
from finlab_crypto.talib_filter import TalibFilter

def crossover(s1, s2):
  if isinstance(s2, numbers.Number):
    return (s1 > s2) & (s1.shift() < s2)
  return (s1 > s2) & (s1.shift() < s2.shift())

def crossunder(s1, s2):
  if isinstance(s2, numbers.Number):
    return (s1 < s2) & (s1.shift() > s2)
  return (s1 < s2) & (s1.shift() > s2.shift())

macd_strategy = TalibStrategy(
    'MACD',
    lambda ohlcv, macd: crossover(macd.macdhist, 0),
    lambda ohlcv, macd: crossunder(macd.macdhist, 0),
)

line_angle_filter = TalibFilter(
    'LINEARREG_ANGLE',
    lambda ohlcv, angle: angle > 0
)

import numpy as np
macd_vars = {
             'fastperiod': np.arange(200, 400, 10),
             'slowperiod': np.arange(200, 400, 10),
             'signalperiod': 10}

angle_vars = {'timeperiod': np.arange(20, 100, 10)}

macd_strategy.backtest(ohlcv, variables=macd_vars,
                       filters={'line_angle': line_angle_filter.create(angle_vars)},
                       freq='1h', plot=True)

In [ ]:
macd_vars = {
    'fastperiod': 320,
    'slowperiod': 300,
    'signalperiod': 10,
}

line_angle_filter_vars = {
    'timeperiod': 20,
}

portfolio = macd_strategy.backtest(
    ohlcv.loc['2019':],
    variables=macd_vars,
    filters={'line_angle': line_angle_filter.create(line_angle_filter_vars)},
    freq='1h', plot=True)